# **Attrition Risk Modeling**
<br/>by Christian Fernandes

Dataset: https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset?resource=download

To leave or not to leave? That is the question many of us in the workforce have probably at least once in our careers considered. Whether it be because we want to chase that next job title or improve our work-life balance there are many reasons why we'd choose to jump ship. 

Well, what does this mean for the employer? In terms of the bottom line, high attrition (voluntary terminations) could engender a number of issues that include but are certainly not limited to higher costs of replacing employees, decreased workplace morale, and potentially, a snowball effect that leads to more and more turnovers. 

What can employers do? They can take advantage of employee data that is tracked in HR systems like Workday and partition it into rosters that capture meaningful employee information such as their job title, current manager, line of business hierarchy, manager hierarchy, salary history, etc. 

Here are some things to consider:
<ul>
    <li> Using geospatial information like addresses, we can pair this data with employee HQ location data to determine the commuting distance for employees. If we pair this with a flag that identifies remote versus in-office employees then we can see how turnover effects the former versus the latter. </li>
    <li> Using employee start dates and the current date, we could determine the length of service for a given employee and see if this could serve as a proxy for a lack of upward mobility. </li>
    <li> Using age information, could it be the case that younger employees are more prone to job hop in today's market? For employees at retirement age, what does turnover look like for this group?</li>
</ul>

## Reading in the data

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from collections import Counter

In [ ]:
empattr = pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv')

In [ ]:
empattr.sample(3)

In [ ]:
empattr.shape

The dataset has 35 variables and 1470 observations. Several things we need to be aware of: 
<ul> 
    <li> Small sample sizes in general can lead to instability in our model parameter estimates </li>
    <li> Ideally, our response variable should be an i.i.d random variable otherwise, any predictions made from this data could be way off.
</ul>


In [ ]:
empattr.dtypes

In [ ]:
# Do we have any values that are null?
if all(empattr.isnull().any() == False): 
    print("Dataset does not have any null values")
else:
    print("There are null values")

In [ ]:
# Find the columns that are considered of object type. 
objcols = [c for c in empattr.columns if empattr[c].dtype == 'O']

In [ ]:
# From the dataset description, these are factor variables
catcols = ["Education", "EnvironmentSatisfaction", "JobInvolvement", "JobSatisfaction", "PerformanceRating", "RelationshipSatisfaction", "WorkLifeBalance"]

In [ ]:
objcols

In [ ]:
# Check to see if any of the object type columns are in the categorical column list defined in the IBM metadata
# Result is an empty set
set(objcols).intersection(set(catcols))

## EDA

In [ ]:
def plot_hist(tibble, colname, response_var):
    tibble[[colname, response_var]].boxplot(by = response_var, grid = False)

def plot_barchart(series_col):
    """a is pd.Series object that names the column to be converted to a barchart"""
    counterDict = dict(Counter(series_col))
    df = pd.DataFrame({'labels': counterDict.keys(), 'values': counterDict.values()})

    df.plot.bar(x = 'labels', y = 'values', title = series_col.name)
    
def plot_figure(tibble, colname, xlab = None, ylab = None, fig_type = None, response_var = None):
    from collections import Counter
    
    if fig_type:
        if fig_type == 'hist':
            tibble[colname].hist(grid = False)
        elif fig_type == 'boxplot':
            plot_hist(tibble, colname, response_var)
        elif fig_type == 'barh':
            plot_barchart(tibble[colname])

In [ ]:
# Descriptive statistics for the categorical data

# First ensure all columns are of the 'object' type - similar to R's as.factor() command
empattr_obj = empattr[objcols + catcols].astype(object)

In [ ]:
empattr_obj.describe()

Already, we can see for some of the observations that a majority of the employees in this dataset did not quit their jobs (Attrition = No). We can also see that a large proportion of observations are male employees (they make up a majority relative to the other gender 'female').   

In [ ]:
printcols = ['Attrition', 'Gender', 'PerformanceRating']
for c in printcols:
    plot_figure(empattr, c,  xlab = c, ylab = None, fig_type = 'barh')

In [ ]:
# Descriptive statistics for the numerical data

# Grabbing all of the columns that were not identified as categorical (not in the empattr_obj dataframe)
empattr_numerical_cols = list(set(empattr.columns.tolist()).difference(set(objcols + catcols)))

In [ ]:
empattr[empattr_numerical_cols].describe()

In [ ]:
fig, ax = plt.subplots(1,3, figsize = (10, 6))
plt.subplot(1,3,1)
plot_figure(empattr, 'Age', 'Age', None, 'hist')
plt.subplot(1,3,2)
plot_figure(empattr, 'YearsSinceLastPromotion', 'YearsSinceLastPromotion', None, 'hist')
plt.subplot(1,3,3)
plot_figure(empattr, 'DistanceFromHome', 'DistanceFromHome', None, 'hist')

In [ ]:
plot_figure(empattr, 'Age', fig_type = 'boxplot', response_var = 'Attrition')

In the age boxplot, it appears that the center of the distribution (as well as the interquartile range) for those who did leave the company seems to hover closer towards those in the late 20s and late 30s age range. 

For those bucketed in the 'No' plot, the retention with respect to age seems to be centered closer towards mid 30s with the interquartile window further up between early 30s and early 40s. 

In [ ]:
plot_figure(empattr, 'YearsSinceLastPromotion', fig_type = 'boxplot', response_var = 'Attrition')

In [ ]:
plot_figure(empattr, 'DistanceFromHome', fig_type = 'boxplot', response_var = 'Attrition')

Boxplots allow us to see the center and spread of our data across groups. In the graphs above, we looked at the relationship of the response, Attrition, with the three predictors of interest to see if any appear to have a noticeable relationship. Age and DistanceFromHome both seem to have slightly different centers and roughly proportional spread (possible relationship?). 

We have to keep in mind, though, that these plots show us the individual bivariate relationship between attrition and each predictor. This implies that if we are building out a model to either describe, explain, or predict attrition in the workplace, we need to be cognizant of the conditional relationship between two or more variables with respect to our response (attrition). For example, an a multiple regression setting (whether it be linear, poisson, logistic, etc.), multicollinearity or near-multicollinearity among predictors say age and years since last promotion could impact the sign, size, and significance of our estimates.  

### Correlation

In [ ]:
# Looking at the correlation among variables (not including EmployeeCount, Standard Hours, or non-numerical columns)
ignore_cols = ['EmployeeCount','StandardHours'] + objcols + catcols
empcor = empattr.drop(columns =ignore_cols).corr()
plt.figure(figsize = (10,10))
sns.heatmap(empcor, annot=True, fmt = '.0%')

## Logistic Regression

Let's suppose that the variables age, YearsSinceLastPromotion, and DistanceFromHome would all provide predictive power  for whether or not an employee will leave the company. 

We can run a logistic regression model using these three variables and assess if the model provides accurate predictions. 

In Logistic Regression, we assume three things, 
<br/>
<ul>
  <li>The logit of our response variable (the log odds) are a linear combination of our predictors and their coefficients</li>
  <li>Our responses are independent random variables</li>
  <li>Our link function is the logit function (the log odds) since it is possible to use other link functions like the probit function etc.</li>
</ul>

### Submodel with full data

In [ ]:
modelformula = 'Attrition ~ Age + YearsSinceLastPromotion + DistanceFromHome'

In [ ]:
smf_model = smf.glm(formula = modelformula, data=empattr, family=sm.families.Binomial())
result = smf_model.fit()
result.summary()

Given an $\alpha$ threshold of 0.05, we can see that all the p-values for the three chosen predictors are indeed less than the threshold and therefore statistically significant.

## Divide up the data intro training and test sets

In [ ]:
# Re-map the Yes's to 1s and No's to 0s
# response = empattr['Attrition'].apply(lambda x: 1 if x.upper() == 'YES' else 0)

In [ ]:
predictors = empattr[['Age', 'YearsSinceLastPromotion', 'DistanceFromHome']]

In [ ]:
response = empattr[['Attrition']]

In [ ]:
# Splitting the dataset into training and testing set
xtrain, xtest, ytrain, ytest = train_test_split(predictors, response, test_size = 0.2, random_state = 0)

In [ ]:
# Merging the training predictor variables back with the training response variables (because out smf.glm() model 
# will be referencing both)
xtrain2 = xtrain.merge(ytrain, left_index = True, right_index = True)

In [ ]:
smf_model_train = smf.glm(formula = modelformula, data=xtrain2, family=sm.families.Binomial())

In [ ]:
smf_model_train_result = smf_model_train.fit()

In [ ]:
smf_model_train_result.summary()

### Testing for overall regression

In [ ]:
from scipy.stats.distributions import chi2

By taking the difference between our null deviance and deviance statistic, we are able to assess whether not the model itself is statistically significant.

Recall that in logistic regression, we maximize the log-likelihood function under the model with no parameters (so intercept only) and we maximize the log-likelihood function under the model with all of the predictors of interest (so Age, DistanceFromHome, and YearsSinceLastPromotion). 

We then take the difference of the two models (so the intercept only model would just be the null deviance) and calculate their differences. This yields a $\chi^2$ statistic with degrees of freedom equal to the number of predictors in our model: 3.

Like the F-test for a multiple linear regression model, the null hypothesis is that all of the coefficient estimates in our full model are 0 versus the alternative hypothesis that **atleast 1** is non-zero.


In [ ]:
# Assuming large data
overallpvalue = chi2.sf(smf_model_train_result.null_deviance - smf_model_train_result.deviance, 3)

print(f"""Our p-value is {overallpvalue}. \n 
This is much lower than the typical alpha threshold of 0.05. At this level, our model is statiscally significant 
because atleast one of the predictors included in the model has explanatory power with respect to Attrition""")
# Alternative way to comput the p(chi^2 > smf_model_train_result.null_deviance - smf_model_train_result.deviance)
# 1-chi2.cdf(smf_model_train_result.null_deviance - smf_model_train_result.deviance, 3)

In [ ]:
# Getting the fitted values from the training set
fitted_xtrain2 = smf_model_train_result.fittedvalues

# Recoding the results (probabilities) into 0 for No and 1 for Yes
fitted_xtrain2_recoded = fitted_xtrain2.apply(lambda x: 0 if x > 0.5 else 1)

In [ ]:
# Getting a quick look at the number of Nos (0) and Yes (1)
Counter(ytrain['Attrition'].to_list())

In [ ]:
# The # of No's and Yes's in the fitted_xtrain_recoded set given our threshold above which was 0.5
Counter(fitted_xtrain2_recoded)

The model with just the three predictors - age, distance from home, and years since last promotion- predicted all of the values to be "no". In other words, it predicted 988/(988+188) or 84% of the values correctly. 

### Making a prediction

In [ ]:
predictions_test = smf_model_train_result.predict(xtest)

In [ ]:
# Apply a threshold 

# Assuming 50/50 chance

th = 0.5

predictions_results = predictions_test.apply(lambda x: 0 if x > th else 1)

In [ ]:
Counter(predictions_results)

In [ ]:
Counter(ytest['Attrition'].to_list())

In [ ]:
predictions_results_recoded = predictions_results.apply(lambda x: 'No' if x == 0 else 'Yes')

In [ ]:
Counter(predictions_results_recoded)

Our model predicted all of the values to be 'No' in this situation (similar to its behavior with our training set). Effectively, that means that our predictions were 83% accurate. In machine learning, we would expect to see similar results as we move from our training sample to our testing sample when we are working with a model with a few predictors. As we add more predictor variables to the dataset, we might end up with a more accurate result during the training phase but suffer from poor performance in the testing phase. 

## Can we do better? - Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Use a random forest classifer, lets start with 50 trees
attr_forest = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)

attr_forest.fit(xtrain, ytrain.values.ravel())


In [ ]:
# What is the accuracy of this model?
attr_forest.score(xtrain, ytrain)

In [ ]:
attr_forest_CM = confusion_matrix(ytest, attr_forest.predict(xtest))
display(attr_forest_CM)

How to read the confusion matrix: 
<table align = "left">
  <tr>
    <td>TN</td>
    <td>FP</td>
  </tr>
    <tr>
    <td>FN</td>
    <td>TP</td>
  </tr>
</table>
</br>
</br>
</br>
<ul>
    <li>The True Positive (TP) value is at position 1,1</li>
<li>The True Negative (TN) value is at position 0,0</li>
<li>The False Negative (FN) value is at position 1,0</li>
<li>The False Positive (FP) value is at position 0,1</li>
</ul>



In [ ]:
tp = attr_forest_CM[1,1]
tn = attr_forest_CM[0,0]

fm_test_acc = (tp + tn)/xtest.shape[0]
print(f"The accuracy of the random forest on the test set is {fm_test_acc}")

### Logistic Regression with sklearn and almost full predictor set

This time we'll look at most of the columns. Recall that EmployeeCount and StandardHours were constants (single level factors that don't really contribute to understanding Attrition).

In [ ]:
newdf = empattr.copy()

In [ ]:
newdf = newdf.drop(columns = ['EmployeeCount','StandardHours'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
recodecols = catcols + objcols
for c in recodecols:
    newdf[c] = LabelEncoder().fit_transform(newdf[c])

In [ ]:
newPreds = newdf.iloc[:, newdf.columns != 'Attrition']
newY = newdf['Attrition']

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(newPreds, newY, test_size = 0.25, random_state = 0)

In [ ]:
# Logistic Regression model using an L1 penalty and Coordinate Descent optimizer
newmodel = LogisticRegression(penalty = 'l1', solver = 'liblinear')

In [ ]:
newdf.head()

In [ ]:
newmodel.fit(xtrain, ytrain)

In [ ]:
newmodel.score(xtrain, ytrain)

In [ ]:
# Predictions
preds = newmodel.predict(xtest) == ytest
recodedpreds = preds.apply(lambda x: 1 if x else 0)

In [ ]:
sum(recodedpreds)/len(recodedpreds)